In [ ]:
import cv2
import numpy as np

def detect_eye_color(image, eyes):
    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the color thresholds for different eye colors
    color_thresholds = {
        'blue': ([90, 50, 20], [130, 255, 255]),  # Lower and upper threshold for blue
        'brown': ([0, 50, 20], [20, 255, 200]),  # Lower and upper threshold for brown
        'black': ([0, 0, 0], [180, 255, 30]),    # Lower and upper threshold for black
        'green': ([50, 50, 20], [80, 255, 200]), # Lower and upper threshold for green
    }

    # Detect the color of each eye based on the color thresholds
    eye_colors = []
    for (x, y, w, h) in eyes:
        eye_region = image[y:y+h, x:x+w]
        hsv_eye = cv2.cvtColor(eye_region, cv2.COLOR_BGR2HSV)
        eye_color = ''
        for color, (lower, upper) in color_thresholds.items():
            lower = np.array(lower, dtype=np.uint8)
            upper = np.array(upper, dtype=np.uint8)
            mask = cv2.inRange(hsv_eye, lower, upper)
            result = cv2.bitwise_and(eye_region, eye_region, mask=mask)
            percentage = np.count_nonzero(result) / (eye_region.shape[0] * eye_region.shape[1]) * 100
            if percentage > 1:
                eye_color = color
                break
        eye_colors.append(eye_color)

    return eye_colors

# Load the image
image = cv2.imread('dataset/eyeimg1.jpeg')

# Detect eyes in the image using Haar cascade
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Detect eye color and write it on the image
eye_colors = detect_eye_color(image, eyes)
for (x, y, w, h), color in zip(eyes, eye_colors):
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(image, color, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image with eye color detection
cv2.imshow('Eye Color Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
